# Feature engineering : Prévision de production solaire à J+1 en temps réel

***Résumé Exécutif de l'Exploration de Données (EDA) - Prévision Solaire J+1***

Cette analyse exploratoire des données (EDA) vise à comprendre les caractéristiques de la production solaire régionale et des variables météorologiques associées, en préparation du développement d'un modèle de prévision à J+1.

**Données** : Dataset horaire sur 2.5 ans (01/2023 - 05/2025) de production RTE et prévisions OpenMétéo. Identification de 2 valeurs manquantes nocturnes dans la cible, qui seront imputées à 0.

**Variable Cible ("solar_mw")** :
- Forte saisonnalité journalière et annuelle, avec une dérive positive due à l'augmentation de la capacité installée ;
- Distribution asymétrique à droite, avec de nombreux zéros (nuit/mauvaises conditions) ; 
- Nécessite une différenciation saisonnière d'ordre 24 pour la stationnarité (confirmée par test ADFuller), avec un ordre AR de 1 (analyse PACF), et un ordre MA de q (à déterminer).

**Variables Explicatives (Covariables) :**
- Fortes corrélations (+0.8) avec la cible : global_tilted_irradiance, shortwave_radiation ;
- Redondances identifiées : Irradiance/Radiation (choix de l'irradiance pour le sens physique), température/température apparente (choix de temperature_2m) ;
- Variables de dispersion (delta_minmax, std) : Essentielles pour capturer l'hétérogénéité spatiale de la région, notamment pour la couverture nuageuse et la DNI ;
- Cross-corrélation : Déphasages clairs pour l'irradiance, l'humidité, la vitesse du vent et la température (justifiant des lags aux ordres 24, 36, 48...). Pression et couverture nuageuse non pertinentes pour les lags.

**Prochaines Étapes (déjà identifiées)**:
- Feature Engineering (encodage cyclique, features laggées, agrégations) ;
- Baseline SARIMAX avec les features physiques les plus corrélées.
- Sélection de features (RFE ou Embedding via LightGBM) ;
- Validation croisée "Expanding Window" avec optimisation des hyperparamètres (Optuna) ;
- Développement de modèles LightGBM et LSTM, avec MC dropout et regression quantile pour quantifier l'incertitude des modèles

#### Contraintes métiers

- Les modèles ensemblistes et DL doivent avoir un temps d'inférence inférieur à la demi-heure, afin de proposer leurs résultats avant la livraison des résultats et en ayant le maximum d'informations possibles (12h) ; 
- **REMINDER TO MYSELF** : Eviter le data leakage sur les moyennes mobiles et les features temporelles ;

#### Analyse comparative modèle statistique linéaire paramétrique/ensembliste/NN

- Le choix d'utiliser le modèle SARIMAX, LightGBM et LSTM est motivé par l'interprétabilité proposé par les deux premiers modèles (Coefficients, Feature importance), avec précision accrue avec le modèle neuronal. Dans la suite des travaux, le modèle LSTM pourra être enrichi par un couche CNN ou un mécanisme d'attention.

#### Plan de configuration technique

- Pipeline reproductible ;
- Stratégie et gestion des données manquantes/outliers (discussions avec métiers, EDA n'a pas constaté d'outliers mais garde-fous conseillés) : Z-score et Isolation Forest proposés
- **REMINDER TO MYSELF** : la gestion de drift, variance sera problématisé ultérieurement, mais à garder en tête lors de la création de ce fichier
- Train/test split par nested CV : 
  - Boucle externe Feature selection (RFE, Feature importance): expanding window cross validation (k=5) ;
    - Boucle interne : expanding window cross validation (k=3) ;
      - Optimisation des hyperparamètres par minimisation de l'AIC pour SARIMAX (maximiser l'interprétabilité). On pourrait aussi proposer une analyse des p-value, mais à relativiser au regard du problème de p-hacking ;
      - Optimisation des hyperparamètres par Optuna pour les modèles LightGBM et LSTM (maximisation de la précision avec minimisation du temps d'entrainement)

### Import des libraries

In [ ]:
# Base
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Structure du signal temporel
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import STL, MSTL
from statsmodels.tsa.stattools import adfuller, kpss

In [5]:
# Confirmation d'être à la racine du dossier
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
os.chdir(project_root)

In [ ]:
# Import data
df = pd.read_csv("data/processed/exploratory_solar_dataset.csv", sep=";", index_col=0)

# Temporal data, indexed
df.index = pd.to_datetime(df.index, utc=True).tz_convert("Europe/Paris")

In [8]:
df.head()

,Solaire (MW),temperature_2m_run_13,sunshine_duration_run_13,is_day_run_13,relative_humidity_2m_run_13,precipitation_run_13,surface_pressure_run_13,cloud_cover_run_13,wind_speed_10m_run_13,wind_direction_10m_run_13,...,direct_normal_irradiance_delta_minmax,direct_normal_irradiance_std,shortwave_radiation_delta_minmax,shortwave_radiation_std,global_tilted_irradiance_delta_minmax,global_tilted_irradiance_std,terrestrial_radiation_delta_minmax,terrestrial_radiation_std,apparent_temperature_delta_minmax,apparent_temperature_std
2023-02-01 01:00:00+01:00,0.0,3.136,0.0,0.0,99.29538,0.0,974.73560,100.0,6.989936,304.50858,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.2179,2.4873
2023-02-01 02:00:00+01:00,0.0,3.436,0.0,0.0,99.64800,0.0,974.41766,100.0,8.089994,302.27563,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.8490,2.3038
2023-02-01 03:00:00+01:00,0.0,3.436,0.0,0.0,99.29709,0.0,974.41766,100.0,9.178235,281.30990,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.6066,1.9406
2023-02-01 04:00:00+01:00,0.0,3.436,0.0,0.0,98.59856,0.1,974.04000,100.0,9.693296,291.80148,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0427,1.7014
2023-02-01 05:00:00+01:00,0.0,3.436,0.0,0.0,99.29709,0.6,974.04000,100.0,10.587918,305.31128,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.8898,1.7487
